<a href="https://colab.research.google.com/github/werd0n4/hand-gesture-classification/blob/master/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

---


In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !ls

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras import initializers

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import os
import cv2
import glob
import random
from matplotlib import pyplot as plt

# Constant parameters

---



In [4]:
horiz_x = 800  
vert_y = 600
#drive paths
#
imgs_train_path = '/content/drive/My Drive/Colab Notebooks/Rozszerzony_dataset/Train/'
imgs_test_path = '/content/drive/My Drive/Colab Notebooks/Rozszerzony_dataset/Test/'
#local paths windows
#
# imgs_train_path = 'C:\\Users\\Werdon\\Google Drive\\Colab Notebooks\\Rozszerzony_dataset\\Train'
# imgs_test_path = 'C:\\Users\\Werdon\\Google Drive\\Colab Notebooks\\Rozszerzony_dataset\\Train'
#local paths manjaro
#
# imgs_train_path = '/home/werdon4/Rozszerzony_dataset/Train'
# imgs_test_path = '/home/werdon4/Rozszerzony_dataset/Test'


In [5]:

class_names = {
    0: "1",
    1: "2",
    2: "3",
    3: "4",
    4: "5",
    5: "A",
    6: "B",
    7: "C",
    8: "D",
    9: "E",
    10: "F",
    11: "G",
    12: "H",
    13: "I",
    14: "K",
    15: "L",
    16: "M",
    17: "N",
    18: "O",
    19: "P",
    20: "R",
    21: "S",
    22: "T",
    23: "U",
    24: "W",
    25: "Y",
    26: "Z"
}


# Auxiliary functions

---


## Image resize

In [6]:

def resize(path):
    img_counter = 0

    for dirname in os.listdir(path): 
        for filename in os.listdir(os.path.join(path, dirname)):
            image_path = os.path.join(path, dirname, filename)
            img = cv2.imread(image_path)
            resized_img = cv2.resize(img, (horiz_x, vert_y))
            cv2.imwrite(image_path, resized_img)
            img_counter += 1
    
    print('Images in set: ' + str(img_counter))


## Image size sanity check

In [7]:

def sanity_check(path):
    counter = 0

    for dirname in os.listdir(path): 
        for filename in os.listdir(os.path.join(path, dirname)):
            image_path = os.path.join(path, dirname, filename)
            img = cv2.imread(image_path)
            if img.shape != (horiz_x, vert_y, 3):
                counter += 1

    print('Sanity result: ' + str(counter))

In [8]:

def show_img(index, X, Y):
    # plt.imshow(X[index])
    plt.imshow(cv2.cvtColor(X[index],cv2.COLOR_BGR2RGB).astype('float32'))
    plt.show()
    nmb = int(np.where(Y[index] == 1)[0])
    print("On image: " + class_names[nmb])

## Load dataset

In [9]:

def load_dataset():
    trainlist = glob.glob(f'{imgs_train_path}/*/*')
    testlist = glob.glob(f'{imgs_test_path}/*/*')
    X_train = np.array( [np.array(cv2.normalize(cv2.imread(fname), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)) for fname in trainlist] )
    X_test = np.array( [np.array(cv2.normalize(cv2.imread(fname), None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)) for fname in testlist] )

    # Y_train = np.array([np.zeros(27) for fname in trainlist])
    Y_train = np.array([0 for fname in trainlist])
    for i,fname in enumerate(trainlist):
        # print(fname)
        img_id = fname.split('/')[7]##7 if windows 4 if manjaro
        img_id = img_id.split('_')[0]
        # Y_train[i][img_id] = 1
        Y_train[i] = img_id


    # Y_test = np.array([np.zeros(27) for fname in testlist])
    Y_test = np.array([0 for fname in testlist])
    for i,fname in enumerate(testlist):
        img_id = fname.split('/')[7]
        img_id = img_id.split('_')[0]
        # Y_test[i][img_id] = 1
        Y_test[i] = img_id

    return X_train, Y_train, X_test, Y_test

# load_dataset()


## Create network model

In [10]:

def create_model():
    model = Sequential()
    # kernel_initializer = initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)
    kernel_initializer=None 
    activ_func = 'relu'
    kernel_regularizer=None
    activity_regularizer=None

    # CONVOLUTIONAL LAYER - 1
    model.add(Conv2D(
        filters=6, 
        kernel_size=(5,5), 
        input_shape=(vert_y, horiz_x, 3), 
        activation=activ_func,
        # kernel_initializer=kernel_initializer
        # kernel_regularizer=kernel_regularizer
        # activity_regularizer=activity_regularizer
    ))

    # POOLING LAYER
    model.add(MaxPool2D(
        pool_size=(2,2),
        strides=(2,2)
    ))

    # CONVOLUTIONAL LAYER - 2
    model.add(Conv2D(
        filters=16, 
        kernel_size=(5,5), 
        activation=activ_func,
        # kernel_initializer=kernel_initializer
        # kernel_regularizer=kernel_regularizer
        # activity_regularizer=activity_regularizer
    ))

    # POOLING LAYER
    model.add(MaxPool2D(
        pool_size=(2,2),
        strides=(2,2)
    ))

    # CONVOLUTIONAL LAYER - 3
    model.add(Conv2D(
        filters=32, 
        kernel_size=(5,5), 
        activation=activ_func,
        # kernel_initializer=kernel_initializer
        # kernel_regularizer=kernel_regularizer
        # activity_regularizer=activity_regularizer
    ))

    # POOLING LAYER
    model.add(MaxPool2D(
        pool_size=(2,2),
        strides=(2,2)
    ))

    # CONVOLUTIONAL LAYER - 4
    model.add(Conv2D(
        filters=120, 
        kernel_size=(5,5), 
        activation=activ_func,
        # kernel_initializer=kernel_initializer
        # kernel_regularizer=kernel_regularizer
        # activity_regularizer=activity_regularizer
    ))

    # POOLING LAYER
    model.add(MaxPool2D(
        pool_size=(2,2),
        strides=(2,2)
    ))

    # CONVOLUTIONAL LAYER - 5
    model.add(Conv2D(
        filters=40, 
        kernel_size=(3,3), 
        activation=activ_func,
        # kernel_initializer=kernel_initializer
        # kernel_regularizer=kernel_regularizer
        # activity_regularizer=activity_regularizer
    ))

    # POOLING LAYER
    model.add(MaxPool2D(
        pool_size=(2,2),
        strides=(2,2)
    ))

    ######## FLATTEN ########
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(84, activation='relu'))
    # model.add(Dense(64, activation='relu'))
    model.add(Dense(27, activation='softmax'))

    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam', 
        metrics=['accuracy']
    )

    return model


# Loading dataset

---


## Resize images and do sanity check

In [11]:
# Uncomment before first run on dataset 
# resize(imgs_train_path)
# sanity_check(imgs_train_path)
# resize(imgs_test_path)
# sanity_check(imgs_test_path)


In [12]:
# X_train, Y_train, X_test, Y_test = load_dataset()

In [13]:
# One hot encoding
# Y_cat_train = to_categorical(Y_train, 27)
# Y_cat_test = to_categorical(Y_test, 27)

## Data augumentation

## Initializing ImageDataGenerator class

In [14]:
image_gen = ImageDataGenerator(rotation_range=5, # rotate the image 20 degrees
                               width_shift_range=0.05, # Shift the pic width by a max of 5%
                               height_shift_range=0.05, # Shift the pic height by a max of 5%
                            #    rescale=1.1, # Rescale the image by normalzing it.
                               shear_range=0.05, # Shear means cutting away part of the image (max 10%)
                               zoom_range=0.05, # Zoom in by 10% max
                               fill_mode='nearest' # Fill in missing pixels with the nearest filled value
                                )
                              

## Augumentation sample result

In [15]:
#augumentation sample result

#original img
# nmb = random.randint(0, 296)
# my_hand = X_test[nmb]
# show_img(nmb, X_test, Y_cat_test)

# #generated img
# gen_img = image_gen.random_transform(my_hand)
# print("Generated image")
# plt.imshow(cv2.cvtColor(gen_img,cv2.COLOR_BGR2RGB).astype('float32'))
# plt.show()

# comparison = my_hand == gen_img
# equal_arrays = comparison.all()
# print("Images are equal?: " + str(equal_arrays))

## Image shape and batch size initialization

In [16]:
image_shape = (horiz_x, vert_y, 3)
batch_size = 16

print(imgs_train_path)
print(imgs_test_path)

/content/drive/My Drive/Colab Notebooks/Rozszerzony_dataset/Train/
/content/drive/My Drive/Colab Notebooks/Rozszerzony_dataset/Test/


## Initializing test and train image geneartors

In [17]:
train_image_gen = image_gen.flow_from_directory(imgs_train_path,
                                               target_size=image_shape[:2],
                                                color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='categorical')

Found 1561 images belonging to 27 classes.


In [18]:
test_image_gen = image_gen.flow_from_directory(imgs_test_path,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='categorical',shuffle=False)

Found 297 images belonging to 27 classes.


In [19]:
#train_image_gen.class_indices

# Create and train model

---


In [20]:
CNN = create_model()
CNN.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 596, 796, 6)       456       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 298, 398, 6)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 294, 394, 16)      2416      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 147, 197, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 143, 193, 32)      12832     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 71, 96, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 67, 92, 120)       9

In [21]:
early_stop = EarlyStopping(monitor='val_loss', patience=3)
#Y_cat_train.shape

## Fit model

In [22]:
# CNN.fit(X_train, Y_cat_train, epochs=12, validation_data=(X_test, Y_cat_test), batch_size=batch_size, callbacks=[early_stop])

## Fit model with data augumentation

In [23]:
results = CNN.fit(train_image_gen, epochs=20, validation_data=test_image_gen, callbacks=[early_stop])

Epoch 1/20
98/98 [==============================] - 185s 2s/step - loss: 3.3432 - accuracy: 0.0448 - val_loss: 3.2871 - val_accuracy: 0.0303
Epoch 2/20
98/98 [==============================] - 185s 2s/step - loss: 3.1966 - accuracy: 0.0839 - val_loss: 3.3959 - val_accuracy: 0.1010
Epoch 3/20
98/98 [==============================] - 184s 2s/step - loss: 3.0831 - accuracy: 0.1179 - val_loss: 3.0100 - val_accuracy: 0.1448
Epoch 4/20
98/98 [==============================] - 183s 2s/step - loss: 2.8334 - accuracy: 0.1717 - val_loss: 2.9145 - val_accuracy: 0.1515
Epoch 5/20
98/98 [==============================] - 181s 2s/step - loss: 2.7156 - accuracy: 0.2159 - val_loss: 2.8042 - val_accuracy: 0.1616
Epoch 6/20
98/98 [==============================] - 181s 2s/step - loss: 2.4780 - accuracy: 0.2582 - val_loss: 2.6612 - val_accuracy: 0.2189
Epoch 7/20
98/98 [==============================] - 181s 2s/step - loss: 2.2492 - accuracy: 0.3261 - val_loss: 2.5455 - val_accuracy: 0.2559
Epoch 8/20
98

## Saving model to file

In [24]:
CNN.save('hand_gesture_classifier.h5')

# Results

---


In [25]:
metrics = pd.DataFrame(CNN.history.history)
metrics

,loss,accuracy,val_loss,val_accuracy
0,3.343206,0.044843,3.287111,0.030303
1,3.196588,0.083921,3.395883,0.101010
2,3.083059,0.117873,3.010034,0.144781
3,2.833420,0.171685,2.914515,0.151515
4,2.715626,0.215887,2.804249,0.161616
5,2.477994,0.258168,2.661197,0.218855
6,2.249238,0.326073,2.545473,0.255892
7,2.059560,0.378603,2.382671,0.276094
8,1.777272,0.459962,2.241129,0.329966
9,1.603663,0.499680,2.456973,0.299663


## Single image prediction

In [26]:
nmb = 2
my_hand = X_test[nmb]
show_img(nmb, X_test, Y_cat_test)

NameError: ignored

In [ ]:
my_hand = my_hand.reshape(1, vert_y, horiz_x, 3)
result = CNN.predict_classes(my_hand)
# result = (CNN.predict(my_hand) > 0.5).astype("int32")
result = int(result)
print('CNN says: ' + class_names[result])

## Predictions for individual classes

In [ ]:
predictions = CNN.predict_classes(X_test)

In [ ]:
print(classification_report(Y_test,predictions))